# Getting Started With TensorFlow

- [From Tensorflow tutorial page](https://www.tensorflow.org/get_started/get_started)

In [1]:
import numpy as np
import tensorflow as tf


### Steps: Recall that in the lecture, the 5 steps for running programs in tensorflow are:
1. Create Tensors (aka variables) that are not yet executed or evaluated.
2. Write operations between those tensors (the so-called computational graph)
3. Initialize your tensors
4. Create a Session
5. Run the Session. 


## Tensors:
- central unit of data in Tensorflow
- a tensor consists of a set of primitive values shaped into an ** array of number of dimensions. ** (Recall when we are doing image reduction, the size is (64,64,3)
- a tensor's ** rank ** is its number of dimensions.'

```python
3 #a rank 0 tensor; this is a scalar with shape []
[1.,2.,3.] # a rank 1 tensor, this is a vector with shape [3]
[[1.,2.,3.],[4.,5.,6.]] #a rank 2 tensor, a matrix with shape [2,3]
[[[1.,2.,3.],[4.,5.,6.]]] # a rank 3 tensor with shape [2,1,3]
```


## Operations and Computational Graphs

- Two discrete sections:
    - **Build** the computational graph
    - **Run** the computational graph

In [4]:
node1 = tf.constant(3.0, dtype = tf.float32)
node2 = tf.constant(4.0,) # implicitly dtype is float32
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


It will not print 3.0 and 4.0 as constants as we would expect, but instead when evaluated, would produce 3.0 and 4.0. To actually evaluate the nodes, **WE MUST RUN THE COMPUTATIONAL GRAPH WITH A SESION.**

There are two ways to create a session that we learned in Deep Learning
1. Method 1
    ```python
    sess = tf.Session() # create the instance
    result = sess.run(..., feed_dict = {...}) # if you have any placeholder that you need to feed
    sess.close() #close the session.
    ```
2. Method2
    ```python
    with tf.Session() as sess: #safer way to run session
        result = sess.run(...,feed_dict = {...})
    ```

In [19]:
sess = tf.Session()
print(sess.run([node1, node2]))
sess.close()

[3.0, 4.0]


In [20]:
# To run this, you have to uncomment the line above

#with tf.Session() as sess:
#    print(sess.run([node1,node2]))

We can further build more complicated computations by combining Tensor nodes with operations(addition, multiplication, etc). ** Operations are also nodes in Tensorflow!** 

In [21]:
sess = tf.Session()
node3 = tf.add(node1,node2)
print("node3:",node3)
print("sess.run(node3):",sess.run(node3))

node3: Tensor("Add_2:0", shape=(), dtype=float32)
sess.run(node3): 7.0


A graph can further be parameterized to accept external inputs, known as ** placeholders**. A placeholder is a promise to provide a value later. (Similar to a parameter in a function)


In [25]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
add_node = a + b
# alternatively
adder_node = tf.add(a,b)
#because we have already created a sess so we could just run it directly
print(sess.run(add_node,feed_dict = {a:3,b:4.5}))
print(sess.run(adder_node,{a:3,b:4.5})) # the keyword "feed_dict" is optional"
print(sess.run(add_node,feed_dict = {a:[1,3], b: [2,4]}))
print(sess.run(adder_node,{a:[1,3], b: [2,4]})) #these should produce the same result

7.5
7.5
[ 3.  7.]
[ 3.  7.]


In [26]:
# To make the computational graph more complex, we could do:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:3, b:4.5}))

22.5


- In Machine Learning, we often need a model that can take arbitrary inputs to make the model trainable. ** Variables ** allow us to add trainable parameters to a graph.They are constructed with ** a type and initial value **

- We see earlier that we use *tf.constant*, and their value can never change. By contrast, Variables are not initialized when you call *tf.Variable *. To initialize all the variables in a tensorflow program, you need to explicitly call a special operation as follow:
```python
init = tf.global_variables_initializer()
sess.run(init)
```

- **init** is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call sess.run, the variables are uninitialized.

In [27]:
W = tf.Variable([.3],dtype = tf.float32)
b = tf.Variable([-.3], dtype = tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

# initialize all variables
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


Now we have created a linear model, but we don't know how good it is. (i.e. we have the yhat, but we need the y to know how far apart is yhat from y). 
Let's use the ** standard loss model for linear regression**, which sums the squares of the deltas between the current model and the provided data.

In [29]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y ) # (yhat - y) ^2
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss,{x:[1,2,3,4], y :[0, -1, -2, -3]}))

23.66


In [31]:
# we could reassign the value of W and b y using tf.assign for any Variable
fixW = tf.assign(W,[-1])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0, -1,-2,-3]}))
# here the cost function reach 0 because we manually change the w and b, 
# but in ml, you would train the model to find that perfect w and b

0.0


## Gradient Descent - ML Optimization Method


In [41]:
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01 is the learning rate alpha, its the hyperparameter
train = optimizer.minimize(loss)

sess.run(init) # this reset values to incorrect default
for i in range(1000): #number of iterations
    sess.run(train,{x:[1,2,3,4], y:[0, -1,-2,-3]})
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


** Notice that the number is very close to -1 and 1 but not really -1 and 1. **

# tf.estimator

- tf.estimator is a higher level Tensorflow library that simplifies the mechanics of ml, that includes:
    - running training loops
    - running evaluation loops
    - managing data sets

In [44]:

#declares list of featuress. We only have one numeric feature here. But there are many
# other types of columns that are more complicated and useful
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_random_seed': 1, '_model_dir': '/tmp/tmpao9qdbrj', '_log_step_count_steps': 100, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpao9qdbrj/model.ckpt.
INFO:tensorflow:step = 1, loss = 23.0
INFO:tensorflow:global_step/sec: 1942.01
INFO:tensorflow:step = 101, loss = 0.277967 (0.053 sec)
INFO:tensorflow:global_step/sec: 2209
INFO:tensorflow:step = 201, loss = 0.0590458 (0.045 sec)
INFO:tensorflow:global_step/sec: 2205.22
INFO:tensorflow:step = 301, loss = 0.0223003 (0.045 sec)
INFO:tensorflow:global_step/sec: 1894.98
INFO:tensorflow:step = 401, loss = 0.00873921 (0.053 sec)
INFO:tensorflow:global_step/sec: 2197.06
INFO:tensorflow:step = 501, loss = 0.0025899 (0.046 sec)
INFO:tensorflow: